<center> ## HUK Coding Challenge </center>

## <br> <center> HUK Coding Challenge </center>
#### <center> (Sebastian Christodoulou) </center>

<center> Auf Englisch geschrieben wegen Terminologie, und weil meiner Erfahrung nach viele Developer das gewohnt sind </center>

<h3>

<u> Contents </u>
* Launching the server with running LLM and API
* Communication: Sending and receiving responses
* Latency, measurements, potential optimizations
* Testing-scenarios with pytest
* A Dive into the software-architechture: Under the hood, and on the surface


</h3>

In [24]:
# !!!!!! ATTENTION !!!!!!!!!
# Please use a venv for installations - else python blocks them
%pip install -U requests



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### <u> Let's start the docker for the "service" </u> 
To be honest, we can barely call this a service. In reality I'd do this 
* in Kubernetes 
* or at least in docker-compose

But to keep compatibility issues minimal, I'll strictly follow the exercise instructions, and just do a docker port-mapping to the host. (port 8123->8123).
This will allow us to send requests to localhost:8123, and reach the server-api that's running on the docker. 
(Again, I'd do this in Kubetnetes)

<u> NOTE </u>: It's may be more reliable to paste these commands below into your shell directly. Don't trust Jupyter's execution of docker


In [14]:
# Pull my deployment docker, specifically made for this task. It has the required tensorflow==2.7.0, and python3.9 [highest possible for tf2.7]
!docker pull sebastianfchr/appl_tfdocker:latest
print("starting the docker. Please wait a bit until the API is running")

!docker run -d --gpus=all -v $(pwd):/code -p 8123:8123 -w /code sebastianfchr/appl_tfdocker:latest -- uvicorn serverapi:app --host 0.0.0.0 --port 8123

# or if you use nerdctl like me:
# nerdctl pull sebastianfchr/appl_tfdocker:latest
# nerdctl run -d --gpus=all -v $(pwd):/code -p 8123:8123 -w /code sebastianfchr/appl_tfdocker:latest -- uvicorn serverapi:app --host 0.0.0.0 --port 8123


latest: Pulling from sebastianfchr/appl_tfdocker
Digest: sha256:d91e8857a1cc7a240a0e38f8362911f19b4f53d314303939b7510299d69ac366
Status: Image is up to date for sebastianfchr/appl_tfdocker:latest
docker.io/sebastianfchr/appl_tfdocker:latest
starting the docker. Please wait a bit until the API is running
0fa5cd082b26b96717ce92044d82bd1647a85b36a9d373be36c9289aac90a981



### <u>Docker image</u>
Above, we use my custom cuda-tf-docker, that I use for such deployments, hosted on docker container-registry `docker.io/sebastianfchr/appl_tfdocker:latest`

It takes some time to download, since it's *cuda enabled*. (It runs tf2.7, and the compatible cuda/cudnn version)

Generally, I'm a fan of lightweigtht docker images. But in this application, where we leverage the full potential of GPU-tensorflow, we have to go with this one.


### Sending requests

All we need here, is the python-package "requests". I've an API endpoint on the server that can be used for
* single sentence prediction
* prediction of "chunks" of sentences 

### API
I has one endpoint:
* `server-url/predict_sentence_batch/` for the chunked version


Since all the functionality is in the docker, we merely need to be able to send requests from pyton. Let's predict some sentiments then


### <center> Important: Please make sure the docker is up and running. Its API-server takes a bit to load </center>

In [15]:
import requests

# docker is reachable through this mapping
url_batch = "http://0.0.0.0:8123/predict_sentence_batch/"

sentences = [
    "going down the beautiful road, I met a horrible rabbit",
    "going down the beautiful road, I met a horrible rabbit",
    "Sadly, the guys from HUK gave me the wrong weights, and I had to do specification training myself",
    "I really hope that despite the whole python compatibility hell you could happily execute everything until here",
    "I really hope that despite the whole python compatibility hell you could happily execute everything until here"
]
sentiments = [
    "positive",
    "negative",
    "negative",
    "negative",
    "positive"
]

# POST-data for the batched sentence
data = {"sentences": [s for s in sentences], "sentiments": [s for s in sentiments]}
response = requests.post(url_batch, json=data, headers={ 'Content-Type': 'application/json' })

try:
    extracted_sentence_fragments = response.json()['data']

    print("\n\n ========================= Sentences, sentiments and predicted fragments ================================\n")
    print("\n".join( ["`{}` \n ==({} sentiment)==> `{}`\n\n".format(t,s,p) for t,s,p in zip(sentences, sentiments, extracted_sentence_fragments)]))


except: 
    print("response seems not to contian json")





 ========================= Sentences, sentiments and predicted fragments ================================

`going down the beautiful road, I met a horrible rabbit` 
 ==(positive sentiment)==> ` beautiful`


`going down the beautiful road, I met a horrible rabbit` 
 ==(negative sentiment)==> ` horrible rabbit`


`Sadly, the guys from HUK gave me the wrong weights, and I had to do specification training myself` 
 ==(negative sentiment)==> ` sadly,`


`I really hope that despite the whole python compatibility hell you could happily execute everything until here` 
 ==(negative sentiment)==> ` hell`


`I really hope that despite the whole python compatibility hell you could happily execute everything until here` 
 ==(positive sentiment)==> ` hope`




### <u> What about latency?</u>

Very good question: This is where it gets interesting. There are some relevant comparisons to make, since we're dealing with a server that takes requests, but we want efficient execution. First, some facts

* GPUS are latency-hiding machines that perform well on "compute-bound", parallelizable programs. ML generally fits this.

* GPUS have a n overhead for kernel-launches. So, launch the kernel on as many predictions (or as our GPU can)

* Server requests have a round-trip time. That adds overhead to every request we make from the client

* The above point means that we profit from chunking predictions within requests. But: Longer requests take a bit longer to send


### A word about speed
When we want to predict a number of sentences, 
we can investigate the time for the following modes.

We differentiate between predictions that go through the API and those that don't:

* single prediction request per API (right, blue)

* chunked prediction requests per API (right, orange)

* single direct prediction on tensorflow (left, blue)

* single chinked prediction on tensorflow (left, orange)



<!-- ![title]("./benchmarks.png") -->

<center><img src="./images/benchmarks.png" width=70% height=70% /></center>

### <u> Why this is relevant </u>

These measurements aren't surprising. But they tell the MLOps engineer something very relevant:
* requests are best chunked before being called by tensorflor (and cuda. We can go into depth here)
* requests are best chunked before being sent to the API to avoid unnecessary round-trip times
* to the above point, we see a small influence of message-size on the round-trip-time of large chunked requests (orange-right) 

###  What the developers can do

In a real a real setting, there will be small requestss per client, and a longer round-trip time.

Thus, the engineer can make some design-improvements in case the servers are under very high load:
* aggregate incoming API-requests from different clients once they reach the server (over a small acceptable timeframe)
* predict them in a chunked fashion (fast!)
* map each prediction back them to the client, and send them back




#### Note: To run the rest, wee need to install requirements_strict.txt:<br> since the Challenge uses `tensorflow 2.7`, we must have a compatible python-version between 2.6 and 2.9. We can use another tf-version of a newer python, but this might run into confilict with transformers! 

requirements_strict.txt represent the requirements demanded in the challenge (i.e. tf2.7 => python3.6-3.9). If you encounter the exception below, try commenting it out and thus installing requirements_relaxed.txt (no guarantees!)


In [25]:
import os
import sys

(py_major, py_minor) = sys.version_info[0:2]

if not (py_major != 2 and (6 <= py_minor <= 9)):
    # NOTE: You can comment this out at your own peril.
    raise Exception('Requires python 2.6-2.9, else you cannot run the code provided by HUK. '
    'You can comment out this exception, and see whether it works with a newer Setup. \n \
     The specification Tensorflow==2.7.0 is only compatible with 2.6-2.9. Similar things hold for transformers ')

    %pip install -r requirements/requirements_relaxed.txt

else: 
    %pip install -r requirements/requirements_strict.txt



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Generate your own benchmarks locally
You can now generate your own benchmarks, with `benchmarks.py`. The code below will generate a `benchmark_tf_and_api_calls.png` in this folder.


<u>GPU Warning</u>: Since the docker is running tf-gpu instance AND the benchmarking-script runs another one, this line will fail if your machine has less than ~4GB GPU vRam (~ 2GB reserved in docker, ~2 GB reserved here). 

In [ ]:
import benchmark

## <u>Python tests: Relevant Test Concepts </u>
1. The interfacing with the tf-model is done correctly
2. The API server behaves correctly

These things are realized in `test_components.py`


##### <u> Test 1. Check for correct tf-model-interfacing and text-prediction:</u>
The text goes through quite some processing before the data gets into the kernel-call. 
* Text preparation 
* Text tokenization 
* Masking
* Prediction, tokenized output
* Decoding of predicted tokens. This gives us the sentence-fragment

I fully automated this process in the below functions 
* for batches of sentences `predict_sentence_batch(<<sentence_list>>, <<sentiment_list>>)` 
* or for single sentences `predict_sentence(<<sentence>>, <<sentiment>>)` 

So that I can predict on text directly. To test this code, I compared
* a completely manually written version of the tokenization-prediction-decoding pipeline 
* my automatized versions 


##### <u> Tests 2. Check for correct API Behavior: </u> 
We want an API that reacts to semantic and structural errors. Following things can go wrong 
* Incorrectly structured POST-Request (Message with wrong fields. We expect it to have 'sentences' and 'sentiments')
* Sentiments are all either ["positive", "negative", "neutral"]
* Sentences or sentiments cannot be empty
* For Batched prediction: One sentiments per sentence  




<u>GPU WARNING</u>: Since the docker is running tf-gpu instance AND the testing runs another tf-gpu instance, this line will fail if your machine has less than ~4GB GPU vRam (~ 2GB occupied per instance). 


In [21]:
# To run all my tests, do this
# it tests all files which start with `test_`. In our case, just `test_components.py`

!pytest

============================= test session starts ==============================
platform linux -- Python 3.7.17, pytest-7.4.4, pluggy-1.2.0
rootdir: /home/seb/Desktop/CodingChallenge_MLE
plugins: anyio-3.7.1
collected 5 items                                                              

test_components.py .....                                                 [100%]

============================== 5 passed in 9.54s ===============================


### <u> Train your own weights </u> 

After refractoring a lot, I only left the training-loop itself intact. Check out the following program to train your own weights. 


<u>Note</u>: I found that this line does not run unless you comply with requirements_strict. <br>It looks like huggingface's transformers relied on tf-calls that changed in later versions (that is, later than about tf2.7)

In [ ]:
import train


### <u> Software-Design and Refractoring </u>

I turned this into the 'roberta' package. The classes 
* `TokenEncoder` and
* `RobertaPredictor`
allow me to be able to write more understandable code

These work on two differnt levels: 
* TokenEncoder prepares input (from any set that follows our training-data conventions) into input_ids_t, attention_mask_t, token_type_ids_t
* RobertaPredictor perform predictions 
    1. on tokens, or 
    2. direclty from text (converts text to tokens, predicts tokens, then decodes)


Here, we use 
1. TokenEncoder to turn data into tokens, and apply the encode-predict-decode pipeline manually (of course, we can always go lower by looking into its member-functions).
2. RobertaPredictor automatizes encoding and prediction. It just takes a sentence and sentiment as arguments.


In [22]:
from roberta import RobertaPredictor, TokenEncoder 
import tokenizers
import pandas as pd

MAX_LEN = 96

tokenizer = tokenizers.ByteLevelBPETokenizer.from_file('./roberta/config/vocab-roberta-base.json', './roberta/config/merges-roberta-base.txt', lowercase=True, add_prefix_space=True) 
rp = RobertaPredictor(MAX_LEN, 'v0-roberta-0.h5', tokenizer)

# 1) RAW VERSION:  
# Use inputs from tokenization, manual tokenized prediction, and decoding of prediction to sentence-fragments
test = pd.read_csv('./data/test.csv').fillna('')
e = TokenEncoder(MAX_LEN, tokenizer)
input_ids_t, attention_mask_t, _ = e.prepare_encode_test(test)
# ids and attention_masks. Predict left end right end of subsequence-range within ids 
ids, ams = input_ids_t[0:5], attention_mask_t[0:5]
ls, rs= rp.predict_tokenized(ids, ams)
# Based on (l, r), obtain the extracted ranges        
predicted_subsequences = [ids[i, l:r+1] for i, (l,r) in enumerate(zip(ls,rs))]
# decode them into sentence sub-fragments via tokenizer
sentence_fragments_predicted_manual = list(map(tokenizer.decode, predicted_subsequences))

print("\n\n ========================= Sentences, sentiments and predicted fragments ================================\n")
print("\n".join( ["`{}` \n ==({} sentiment)==> `{}`\n\n".format(t,s,p) for t,s,p in zip(test['text'],test['sentiment'], sentence_fragments_predicted_manual)]))




All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /home/seb/Desktop/CodingChallenge_MLE/roberta/config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.




 ========================= Sentences, sentiments and predicted fragments ================================

`Last session of the day  http://twitpic.com/67ezh` 
 ==(neutral sentiment)==> ` last session of the day http://twitpic.com/67ezh</s></s>`


` Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).` 
 ==(positive sentiment)==> ` shanghai is also really exciting (precisely -- skyscrapers gal`


`Recession hit Veronique Branquinho, she has to quit her company, such a shame!` 
 ==(negative sentiment)==> `ique branquinho, she has to quit`


` happy bday!` 
 ==(positive sentiment)==> ` happy b`


` http://twitpic.com/4w75p - I like it!!` 
 ==(positive sentiment)==> `pic.com/4w75p`




#### The above is just what's happening under the hood. As promised, my software does this via much simpler calls, since RobertaPredictor can predict from text directly (and performs the intermediate steps automatically).

In [23]:
test = pd.read_csv('./data/test.csv').fillna('')

sentences_fragments_automatic = rp.predict_sentence_batch(test['text'][0:5], test['sentiment'][0:5])


print("\n\n ========================= Sentences, sentiments and predicted fragments ================================\n")
print("\n".join( ["`{}` \n ==({} sentiment)==> `{}`\n\n".format(t,s,p) for t,s,p in zip(test['text'],test['sentiment'], sentences_fragments_automatic)]))





 ========================= Sentences, sentiments and predicted fragments ================================

`Last session of the day  http://twitpic.com/67ezh` 
 ==(neutral sentiment)==> ` last session of the day http://twitpic.com/67ezh</s></s>`


` Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).` 
 ==(positive sentiment)==> ` shanghai is also really exciting (precisely -- skyscrapers gal`


`Recession hit Veronique Branquinho, she has to quit her company, such a shame!` 
 ==(negative sentiment)==> `ique branquinho, she has to quit`


` happy bday!` 
 ==(positive sentiment)==> ` happy b`


` http://twitpic.com/4w75p - I like it!!` 
 ==(positive sentiment)==> `pic.com/4w75p`


